In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AffinityPropagation
from collections import Counter
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram


In [193]:
pd.options.display.max_colwidth = 100

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from textblob.sentiments import NaiveBayesAnalyzer
import re
import pandas as pd
import sqlite3
# from textblob import TextBlob

import seaborn as sns
sns.set(color_codes=True)

# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

In [ ]:
cd Desktop/

In [ ]:
df = pd.read_csv('../pitchfork3.csv')

In [367]:
df = pd.read_csv('../pitchfork3.csv')

years = np.arange(1999, 2018)
df_new = df[df['new_album'] == 1]
df_reissue = df[df['new_album'] == 0]

df_spotify = pd.read_csv('spotify_with_rank.csv')

df_merged = df.merge(df_spotify, how='left', on=['artist', 'title'])

In [368]:
# 4. Limit the number of features and see if the words of the topics change.
vectorizer = TfidfVectorizer(stop_words='english', max_features=500, max_df=.4)
X = vectorizer.fit_transform(df_merged['content'].values.astype('U'))

In [369]:
df_merged['tf-idf'] = list(X.toarray())

In [370]:
df_merged['danceability rounded'] = (df_merged['danceability mean']*10).round(1)
df_merged['valence rounded'] = (df_merged['valence mean']*10).round(1)
df_merged['energy rounded'] = (df_merged['energy mean']*10).round(1)
df_merged['acousticness rounded'] = (df_merged['acousticness mean']*10).round(1)
df_merged['score_bin_5'] = pd.cut(df_merged['score'], bins=5, labels=False)

In [372]:
df_merged['danceability rounded'].fillna(value=df_merged['danceability rounded'].mean(), inplace=True)
df_merged['valence rounded'].fillna(value=df_merged['valence rounded'].mean(), inplace=True)
df_merged['energy rounded'].fillna(value=df_merged['energy rounded'].mean(), inplace=True)
df_merged['acousticness rounded'].fillna(value=df_merged['acousticness rounded'].mean(), inplace=True)





In [386]:
df_merged['spotify_match'] = np.where(df_merged['year sum'] > 0, 1, 0)


/Users/andrewbashford/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [364]:
df_merged['acousticness rounded'].mean()

2.939579640415288

In [340]:
import graphlab

In [341]:
df_merged.shape

(19867, 98)

In [387]:
df_merged.head()

,Unnamed: 0_x,Unnamed: 0.1,index,reviewid,title,artist,url,score,best_new_music,best_new_reissue,...,year mean,year std,order_of_release,tf-idf,danceability rounded,valence rounded,energy rounded,acousticness rounded,score_bin_5,spotify_match
0,0,0,22,5a3aa14cdea49a0a938ccb18,merry christmas lil mama: re-wrapped,"chance the rapper, jeremih",https://pitchfork.com/reviews/albums/merry-christmas-lil-mama-re-wrapped/,7.7,0,0,...,NaN,NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0492206790311, 0.0, 0.0599015053475, 0.122856726748, 0.0, 0.0, ...",4.803942,3.948101,6.143214,2.939365,3,0
1,1,1,23,5a2eacf785030943c5158845,pressure,jeezy,https://pitchfork.com/reviews/albums/pressure/,6.3,0,0,...,2017.0,0.0,21.0,"[0.0, 0.0, 0.0, 0.0, 0.088606767019, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.164670809873, 0.0...",6.400000,3.400000,6.900000,0.400000,3,1
2,2,2,24,5a0a0f9fd5ec0a321fb119a2,the beautiful & damned,g-eazy,https://pitchfork.com/reviews/albums/g-eazy-the-beautiful-and-damned/,5.1,0,0,...,2017.0,0.0,5.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.600000,3.300000,6.600000,1.300000,2,1
3,3,3,25,5a285f957d91d040cf1e35e1,quiet storm,shy glizzy,https://pitchfork.com/reviews/albums/shy-glizzy-quiet-storm/,7.1,0,0,...,2017.0,0.0,6.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.093...",7.900000,4.400000,6.500000,0.800000,3,1
4,4,4,26,5a2eb9d5a31af72b3b598039,pop 2,charli xcx,https://pitchfork.com/reviews/albums/charli-xcx-pop-2/,8.4,1,0,...,2017.0,0.0,4.0,"[0.127553030837, 0.0, 0.0636399669879, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0636399669879, 0.090...",6.100000,4.000000,6.100000,1.900000,4,1


In [380]:
df_merged =df_merged.dropna(subset=['content'])

In [388]:
sf = graphlab.SFrame(df_merged[['url', 'tf-idf', 
                                'danceability rounded', 'valence rounded', 'energy rounded', 'acousticness rounded', 
                                'pub_year', u'genre_electronic', u'genre_experimental',
       u'genre_folk/country', u'genre_global', u'genre_jazz', u'genre_metal',
       u'genre_pop/r&b', u'genre_rap', u'genre_rock', 'new_album', 'spotify_match']])

In [389]:
recommender = graphlab.recommender.item_content_recommender.create(sf, item_id='url')

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['tf-idf', 'danceability rounded', 'valence rounded', 'energy rounded', 'acousticness rounded', 'pub_year', 'genre_electronic', 'genre_experimental', 'genre_folk/country', 'genre_global', 'genre_jazz', 'genre_metal', 'genre_pop/r&b', 'genre_rap', 'genre_rock', 'new_album', 'spotify_match']
Excluded Features : ['url']

Column                Type   Interpretation  Transforms   Output Type
--------------------  -----  --------------  -----------  -----------
tf-idf                array  vector          None         array      
danceability rounded  float  numerical       None         float      
valence rounded       float  numerical       None         float      
energy rounded        float  numerical       None         float      
acousticness rounded  float  numerical       None         float      
pub_year              int    numerical       None         int        
genre_electr

Recsys training: model = item_content_recommender

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

...

Data has 0 observations with 0 users and 19619 items.

Data prepared in: 1.81126s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.087441s

In [395]:
def get_similar(df, url, recommender):
    top_5_similar = recommender.get_similar_items([url], k=20)['similar']
    return df[(df['score'] > 7) & 
               (df['url'].isin(top_5_similar))][['artist','title', 'score', 'url', 'danceability rounded',
                                                'valence rounded', 'energy rounded', 'acousticness rounded', 
                                               'pub_year', 'spotify_match']].head(10)

In [398]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/14880-my-beautiful-dark-twisted-fantasy/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
5287,earl sweatshirt,doris,8.3,https://pitchfork.com/reviews/albums/18380-earl-sweatshirt-doris/,5.5,4.7,6.6,1.8,2013,1
5497,kanye west,yeezus,9.5,https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/,5.6,3.5,5.9,1.7,2013,1
6249,meek mill,dreams and nightmares,7.4,https://pitchfork.com/reviews/albums/17315-dreams-and-nightmares/,5.3,3.9,7.7,1.2,2012,1
6282,kendrick lamar,"good kid, m.a.a.d city",9.5,https://pitchfork.com/reviews/albums/17253-good-kid-maad-city/,5.8,3.2,6.6,1.2,2012,1
6548,troy ave,bricks in my backpack 3,7.5,https://pitchfork.com/reviews/albums/16932-bricks-in-my-backpack-3/,5.5,4.6,7.4,2.1,2012,1
8001,"tyler, the creator",goblin,8.0,https://pitchfork.com/reviews/albums/15413-goblin/,5.7,3.8,5.8,1.3,2011,1
8991,kelis,flesh tone,7.2,https://pitchfork.com/reviews/albums/14439-flesh-tone/,5.4,3.6,6.7,2.0,2010,1
9071,drake,thank me later,8.4,https://pitchfork.com/reviews/albums/14367-thank-me-later/,5.4,4.7,7.5,2.0,2010,1
10132,willie isz,georgiavania,7.7,https://pitchfork.com/reviews/albums/13275-georgiavania/,5.3,4.0,6.4,0.9,2009,1
10212,drake,so far gone,7.4,https://pitchfork.com/reviews/albums/13238-so-far-gone/,5.9,3.8,6.8,2.7,2009,1


In [399]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/17253-good-kid-maad-city/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
2708,g herbo,ballin like im kobe,7.2,https://pitchfork.com/reviews/albums/21153-ballin-like-im-kobe/,5.8,3.5,6.6,1.2,2015,1
3070,meek mill,dreams worth more than money,7.4,https://pitchfork.com/reviews/albums/20778-dreams-worth-more-than-money/,6.3,3.5,6.5,1.4,2015,1
3191,a$ap rocky,at.long.last.a$ap,7.8,https://pitchfork.com/reviews/albums/20661-atlonglastaap/,6.0,2.9,6.0,1.9,2015,1
3837,run the jewels,run the jewels 2,9.0,https://pitchfork.com/reviews/albums/19968-run-the-jewels-run-the-jewels-2/,6.2,3.3,6.4,1.0,2014,1
3962,"freeway, the jacka",highway robbery,7.9,https://pitchfork.com/reviews/albums/19743-freeway-the-jacka-highway-robbery/,6.0,4.0,6.5,0.7,2014,1
5101,pusha t,my name is my name,8.0,https://pitchfork.com/reviews/albums/18277-pusha-t-my-name-is-my-name/,5.8,4.3,6.9,1.1,2013,1
5164,drake,nothing was the same,8.6,https://pitchfork.com/reviews/albums/18511-drake-nothing-was-the-same/,6.6,3.3,5.7,1.5,2013,1
5497,kanye west,yeezus,9.5,https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/,5.6,3.5,5.9,1.7,2013,1
6871,future,pluto,7.8,https://pitchfork.com/reviews/albums/16554-pluto/,6.3,4.0,6.5,0.8,2012,1
7398,nacho picasso,for the glory,7.9,https://pitchfork.com/reviews/albums/15976-for-the-glory/,6.6,3.3,5.9,1.6,2011,1


In [400]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/19968-run-the-jewels-run-the-jewels-2/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
1237,g herbo,strictly 4 my fans,7.5,https://pitchfork.com/reviews/albums/22758-strictly-4-my-fans/,6.1,3.5,6.6,0.9,2017,1
1965,vic mensa,theres alot going on,7.5,https://pitchfork.com/reviews/albums/21998-theres-alot-going-on/,6.5,3.4,6.5,1.7,2016,1
2708,g herbo,ballin like im kobe,7.2,https://pitchfork.com/reviews/albums/21153-ballin-like-im-kobe/,5.8,3.5,6.6,1.2,2015,1
3070,meek mill,dreams worth more than money,7.4,https://pitchfork.com/reviews/albums/20778-dreams-worth-more-than-money/,6.3,3.5,6.5,1.4,2015,1
3139,boosie badazz,touch down 2 cause hell,7.4,https://pitchfork.com/reviews/albums/20670-touch-down-2-cause-hell/,6.4,3.9,7.1,1.2,2015,1
3191,a$ap rocky,at.long.last.a$ap,7.8,https://pitchfork.com/reviews/albums/20661-atlonglastaap/,6.0,2.9,6.0,1.9,2015,1
3962,"freeway, the jacka",highway robbery,7.9,https://pitchfork.com/reviews/albums/19743-freeway-the-jacka-highway-robbery/,6.0,4.0,6.5,0.7,2014,1
5164,drake,nothing was the same,8.6,https://pitchfork.com/reviews/albums/18511-drake-nothing-was-the-same/,6.6,3.3,5.7,1.5,2013,1
5451,run the jewels,run the jewels,8.5,https://pitchfork.com/reviews/albums/18219-run-the-jewels-run-the-jewels/,6.8,3.4,7.5,0.9,2013,1
5497,kanye west,yeezus,9.5,https://pitchfork.com/reviews/albums/18172-kanye-west-yeezus/,5.6,3.5,5.9,1.7,2013,1


In [406]:
get_similar(df_merged, 'https://pitchfork.com/reviews/albums/17064-the-disintegration-loops/', recommender)

,artist,title,score,url,danceability rounded,valence rounded,energy rounded,acousticness rounded,pub_year,spotify_match
1581,biosphere,departed glories,7.6,https://pitchfork.com/reviews/albums/22302-departed-glories/,2.1,0.9,2.1,9.0,2016,1
1655,william basinski,92982,8.7,https://pitchfork.com/reviews/albums/22277-william-basinski-92982/,1.8,0.4,2.0,9.6,2016,1
2553,bing & ruth,city lake,7.6,https://pitchfork.com/reviews/albums/21256-city-lake/,1.7,0.8,1.1,9.2,2015,1
2869,masayoshi fujita,apologues,8.2,https://pitchfork.com/reviews/albums/20931-apologues/,2.6,0.6,1.2,9.9,2015,1
2953,laraaji,lotus collage,8.0,https://pitchfork.com/reviews/albums/20839-all-in-one-peace/,3.1,2.0,3.1,8.5,2015,1
3707,jordan de la sierra,gymnosphere: song of the rose,7.5,https://pitchfork.com/reviews/albums/19994-gymnosphere-song-of-the-rose/,1.5,1.5,1.5,9.9,2014,1
3856,bing & ruth,tomorrow was the golden age,8.1,https://pitchfork.com/reviews/albums/19887-bing-ruth-tomorrow-was-the-golden-age/,1.6,0.9,1.1,8.7,2014,1
3879,a winged victory for the sullen,atomos,7.3,https://pitchfork.com/reviews/albums/19868-a-winged-victory-for-the-sullen-atomos/,1.8,0.7,1.6,8.9,2014,1
4464,william basinski,melancholia,7.4,https://pitchfork.com/reviews/albums/19246-william-basinski-melancholia/,2.3,0.7,0.6,9.8,2014,1
4613,bo anders persson,love is here to stay,7.6,https://pitchfork.com/reviews/albums/19116-bo-anders-persson-love-is-here-to-stay/,2.0,1.1,1.4,9.7,2014,1


In [286]:
df[df['artist'] == 'foals'].columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'index', u'reviewid', u'title',
       u'artist', u'url', u'score', u'best_new_music', u'best_new_reissue',
       u'author', u'pub_date', u'pub_weekday', u'pub_day', u'pub_month',
       u'pub_year', u'year', u'genre_electronic', u'genre_experimental',
       u'genre_folk/country', u'genre_global', u'genre_jazz', u'genre_metal',
       u'genre_pop/r&b', u'genre_rap', u'genre_rock', u'content', u'abstract',
       u'reissue', u'num_years_since_release', u'new_album', u'abstract_adj',
       u'content_adj', u'cont_polarity', u'cont_subjectivity', u'abs_polarity',
       u'abs_subjectivity', u'abs_adj_polarity', u'abs_adj_subjectivity',
       u'cont_adj_polarity', u'cont_adj_subjectivity', u'word_count',
       u'adj_count', u'adj_freq', u'score_bin', u'avg_word_length',
       u'adj_word_length', u'num_prev_reviews', u'avg_score_of_prev_reviews',
       u'number_prev_bnm', u'era'],
      dtype='object')